In [ ]:
import pandas as pd
from time import time
!ls

import pyarrow.parquet as pq

pq_file = pq.ParquetFile("green_tripdata_2019-01.parquet")
print(pq_file.schema)

In [ ]:
import pyarrow.parquet as pq

table = pq.read_table("green_tripdata_2019-01.parquet")
green_trips_df = table.to_pandas()
green_trips_df.head(10)

In [ ]:
zone_df = pd.read_csv("taxi_zone_lookup.csv")
zone_df.shape

In [ ]:
from sqlalchemy import create_engine

PG_user = "lana"
PG_pass = "1234"
PG_host = "localhost"
PG_port = 5432
PG_db = "ny_taxi"

engine = create_engine(f"postgresql://{PG_user}:{PG_pass}@{PG_host}:{PG_port}/{PG_db}")
con = engine.connect()


In [ ]:
con

In [ ]:
# Generator function to read parquet in chunks
def read_parquet_in_chunks(file_path, chunksize=100000):
    parquet_file = pq.ParquetFile(file_path)
    for batch in parquet_file.iter_batches(batch_size=chunksize):
        yield batch.to_pandas()

# Iterator (mimics read_csv iterator)
df_iter = read_parquet_in_chunks("green_tripdata_2019-01.parquet", chunksize=10000)

# Load first chunk
df = next(df_iter)

# Convert datetime columns
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

# Create table schema in SQL database (replace with your SQLAlchemy connection object)
df.head(0).to_sql(con=con, name="green_trip_data", if_exists="replace", index=False)



In [ ]:
df.to_sql(con=con, name="green_trip_data", if_exists="append", index=False)

In [ ]:
try:
    while True:
        t_start = time()
        df = next(df_iter)

        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.to_sql(con=con, name="green_trip_data", if_exists="append", index=False)

        end_time = time()
        print("New records added {}".format(end_time - t_start))

except StopIteration:
    print("All record imported or ingested")

